# Advent of Code

## 2024-012-015
## 2024 015

https://adventofcode.com/2024/day/15

In [5]:
def solve():
    # Read the input file
    with open('input.txt', 'r') as file:
        lines = [line.rstrip('\n') for line in file.readlines()]

    # Separate out the map portion vs. the moves portion
    # --------------------------------------------------
    #
    # According to the puzzle, the map is 50 lines high (each line 50+ columns wide)
    # enclosed by '##################################################' on top and bottom.
    # After the map, there's a line break or two, then an enormous move string that may
    # span multiple lines. We need to parse them carefully.

    warehouse_map = []
    move_lines = []
    parsing_map = True
    for line in lines:
        # End of map when we see a line that starts with something other than '#' or is blank,
        # or when we suspect the warehouse is fully read (since it’s a 50x50 or similar).
        if parsing_map:
            if line.strip() == '' or not line.startswith('#'):
                parsing_map = False
            else:
                warehouse_map.append(line)
        else:
            # Now we’re in the moves portion; puzzle states newlines in the move string are ignored
            move_lines.append(line.strip())

    # Combine all move-lines into one big string of moves
    moves = ''.join(move_lines)

    # Dimensions of the map
    height = len(warehouse_map)
    width  = len(warehouse_map[0]) if height > 0 else 0

    # Parse the map into a data structure
    # -----------------------------------
    boxes = set()     # set of (r, c) positions for boxes
    walls = set()     # set of (r, c) positions for walls
    robot = None      # (r, c) for robot

    for r, row in enumerate(warehouse_map):
        for c, ch in enumerate(row):
            if ch == '#':
                walls.add((r, c))
            elif ch == 'O':
                boxes.add((r, c))
            elif ch == '@':
                robot = (r, c)
            # We ignore '.' and everything else for the floor.

    # Helper for performing a move
    # ----------------------------
    # We want to move the robot in direction (dr, dc).
    # Return updated (robot, boxes).

    def attempt_move(robot_rc, boxes, dr, dc):
        (rr, cc) = robot_rc
        nr, nc = rr + dr, cc + dc  # new robot position if possible

        # If new robot spot is a wall, move blocked, do nothing
        if (nr, nc) in walls:
            return robot_rc, boxes

        # If new robot spot is empty floor, just move
        if (nr, nc) not in boxes:
            return (nr, nc), boxes

        # Else, (nr,nc) has a box. We need to push that box chain if possible.
        # Let’s gather all consecutive boxes in this direction (like a chain).
        # Start from the front-most box, going in the pushing direction, find how far the chain goes.

        # chain will hold all boxes in the line (nr, nc), (nr+dr, nc+dc), (nr+2*dr, nc+2*dc), ...
        chain = []
        check_r, check_c = nr, nc
        while (check_r, check_c) in boxes:
            chain.append((check_r, check_c))
            check_r += dr
            check_c += dc

        # Now check_r, check_c is the tile *after* the last box in the chain
        # If that tile is a wall or another box, we can’t push.
        if (check_r, check_c) in walls or (check_r, check_c) in boxes:
            # push fails, robot can’t move
            return robot_rc, boxes

        # Otherwise, we can push successfully.  We move the last box in chain forward by one,
        # then the second-last into the last's old location, etc.
        new_boxes = set(boxes)

        # Move the chain from last to first so we don’t overwrite
        for i in reversed(range(len(chain))):
            (br, bc) = chain[i]  # box row/col
            new_boxes.remove((br, bc))
            new_boxes.add((br + dr, bc + dc))

        # Finally, robot steps into the position vacated by the first box in the chain
        new_robot = (nr, nc)
        return new_robot, new_boxes

    # Directions map
    dir_map = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1),
    }

    # Simulate all moves
    # ------------------
    rpos = robot
    bset = boxes
    for m in moves:
        dr, dc = dir_map[m]
        rpos, bset = attempt_move(rpos, bset, dr, dc)

    # Compute final sum of GPS coordinates
    # ------------------------------------
    # GPS(row, col) = 100*row + col
    # (Distance from top edge = row index, distance from left edge = col index)

    total = 0
    for (r, c) in bset:
        total += 100*r + c

    print(total)

if __name__ == "__main__":
    solve()


1552879


In [1]:
import sys

ans = 0

st = False
moves = ""
origgrid = []

# Parse input from file
with open("input.txt", "r") as file:
    for line in file:
        if line.strip() == "":
            st = True
            continue
        if st:
            moves += line.strip()
        else:
            origgrid.append(list(line.strip()))

N = len(origgrid)
M = len(origgrid[0])
cur_loc = None

grid = [[] for _ in range(N)]
for i in range(N):
    for j in range(M):
        if origgrid[i][j] == "#":
            grid[i].append("#")
            grid[i].append("#")
        elif origgrid[i][j] == "O":
            grid[i].append("[")
            grid[i].append("]")
        elif origgrid[i][j] == ".":
            grid[i].append(".")
            grid[i].append(".")
        elif origgrid[i][j] == "@":
            cur_loc = (i, len(grid[i]))
            grid[i].append("@")
            grid[i].append(".")

N = len(grid)
M = len(grid[0])
for i in range(N):
    print("".join(grid[i]))

symtodir = {">": (0,1), "<": (0,-1), "^": (-1,0), "v": (1,0)}

def affecting(x, y):
    if grid[x][y] == ".":
        return set()
    elif grid[x][y] == "[":
        return {(x, y), (x, y+1)}
    elif grid[x][y] == "]":
        return {(x, y), (x, y-1)}
    
for move in moves:
    dir = symtodir[move]
    newloc = (cur_loc[0]+dir[0], cur_loc[1]+dir[1])
    if dir[0] == 0 or grid[newloc[0]][newloc[1]] == ".":
        finalloc = newloc
        while grid[finalloc[0]][finalloc[1]] not in {".", "#"}:
            finalloc = (finalloc[0]+dir[0], finalloc[1]+dir[1])

        if grid[finalloc[0]][finalloc[1]] != "#":
            assert grid[newloc[0]][newloc[1]] != "#"
            while (finalloc[0]-dir[0], finalloc[1]-dir[1]) != cur_loc:
                grid[finalloc[0]][finalloc[1]] = grid[finalloc[0]-dir[0]][finalloc[1]-dir[1]]
                finalloc = (finalloc[0]-dir[0], finalloc[1]-dir[1])
            grid[newloc[0]][newloc[1]] = "@"
            grid[cur_loc[0]][cur_loc[1]] = "."
            cur_loc = newloc
    elif grid[newloc[0]][newloc[1]] == "#":
        pass
    else:
        affectings = [affecting(newloc[0], newloc[1])]
        bad = False
        while len(affectings[-1]) > 0:
            new_affect = set()
            for x, y in affectings[-1]:
                if grid[x+dir[0]][y] == "#":
                    bad = True
                    break
                for xd, yd in affecting(x+dir[0], y):
                    new_affect.add((xd, yd))
            if bad:
                break
            affectings.append(new_affect)

        if not bad:
            for i in range(len(affectings)-1, -1, -1):
                for x, y in affectings[i]:
                    grid[x+dir[0]][y] = grid[x][y]
                    grid[x][y] = "."
            grid[newloc[0]][newloc[1]] = "@"
            grid[cur_loc[0]][cur_loc[1]] = "."
            cur_loc = newloc

for i in range(N):
    print("".join(grid[i]))
    for j in range(M):
        if grid[i][j] == "[":
            ans += 100*i+j
        
print(ans)

####################################################################################################
##[]##......[]......[]....##........[]..[][][]....##......................[]..##....##..[]....##..##
##......[]..##..[]........[]..[]##..........##[][]........[]........................[]............##
##....[]..##..[][]..[]..............##..[]##[]....##....[]..[]......##..[][]..[][]..[][][]##[]..[]##
##....[]....[]..[]....[]..[]......[][]..[]..[][]..##..####........[]..[]##..[]......##[]....[]....##
##....[]..[]........[]......[]......[]..[][][]..[]..[]..........##..[][]..[]..[]..##[]..[][]..[]..##
##..........[]....##............##............[]..........##........##....[]##[]............[]....##
##..........[][][][]..................[][][]..[]..[]..[]......[][]..[]..............[]..........[]##
####..[]....##..[]..............##..........[][]..............[]......##..................[][]....##
##..[]..##........####....[]..[]....[]##........[][]..............[]....[][]##[][]........#

In [25]:
def read_input(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Split map and movement instructions
    map_data = []
    moves = ""
    is_map = True

    for line in lines:
        if line.strip() == "":
            continue
        if line.strip()[0] in "<>^v":
            is_map = False

        if is_map:
            map_data.append(line.rstrip())
        else:
            moves += line.strip()

    return map_data, moves


def parse_map(map_data):
    warehouse = []
    robot_position = None

    for y, row in enumerate(map_data):
        row_data = []
        for x, char in enumerate(row):
            if char == '@':
                robot_position = (y, x)
                row_data.append('@')  # Keep the robot in place for visualization
            else:
                row_data.append(char)
        warehouse.append(row_data)

    return warehouse, robot_position


def scale_up_map(warehouse):
    # Create a new scaled warehouse with dimensions 2x width and same height
    scaled_warehouse = [[" " for _ in range(len(warehouse[0]) * 2)] for _ in range(len(warehouse))]

    for y, row in enumerate(warehouse):
        for x, char in enumerate(row):
            # Map characters to their scaled counterparts
            if char == '#':
                scaled_warehouse[y][x * 2] = '#'
                scaled_warehouse[y][x * 2 + 1] = '#'
            elif char == 'O':
                scaled_warehouse[y][x * 2] = '['
                scaled_warehouse[y][x * 2 + 1] = ']'
            elif char == '.':
                scaled_warehouse[y][x * 2] = '.'
                scaled_warehouse[y][x * 2 + 1] = '.'
            elif char == '@':
                scaled_warehouse[y][x * 2] = '@'
                scaled_warehouse[y][x * 2 + 1] = '.'

    return scaled_warehouse


def print_scaled_warehouse(warehouse):
    for row in warehouse:
        print(''.join(row))


if __name__ == "__main__":
    input_file = "sample-input.txt"  # Replace with the path to your sample input file
    map_data, moves = read_input(input_file)
    warehouse, robot_position = parse_map(map_data)
    scaled_warehouse = scale_up_map(warehouse)

    print("\nScaled Warehouse State:")
    print_scaled_warehouse(scaled_warehouse)


Scaled Warehouse State:
####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##....[]@.....[]..##
##[]##....[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################


In [28]:
def read_input(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Split map and movement instructions
    map_data = []
    moves = ""
    is_map = True

    for line in lines:
        if line.strip() == "":
            continue
        if line.strip()[0] in "<>^v":
            is_map = False

        if is_map:
            map_data.append(line.rstrip())
        else:
            moves += line.strip()

    return map_data, moves


def parse_map(map_data):
    warehouse = []
    robot_position = None

    for y, row in enumerate(map_data):
        row_data = []
        for x, char in enumerate(row):
            if char == '@':
                robot_position = (y, x)
                row_data.append('@')  # Keep the robot in place for visualization
            else:
                row_data.append(char)
        warehouse.append(row_data)

    return warehouse, robot_position


def scale_up_map(warehouse):
    # Create a new scaled warehouse with dimensions 2x width and same height
    scaled_warehouse = [[" " for _ in range(len(warehouse[0]) * 2)] for _ in range(len(warehouse))]

    for y, row in enumerate(warehouse):
        for x, char in enumerate(row):
            # Map characters to their scaled counterparts
            if char == '#':
                scaled_warehouse[y][x * 2] = '#'
                scaled_warehouse[y][x * 2 + 1] = '#'
            elif char == 'O':
                scaled_warehouse[y][x * 2] = '['
                scaled_warehouse[y][x * 2 + 1] = ']'
            elif char == '.':
                scaled_warehouse[y][x * 2] = '.'
                scaled_warehouse[y][x * 2 + 1] = '.'
            elif char == '@':
                scaled_warehouse[y][x * 2] = '@'
                scaled_warehouse[y][x * 2 + 1] = '.'

    return scaled_warehouse


def simulate_robot(warehouse, robot_position, moves):
    directions = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -2),
        '>': (0, 2)
    }

    for move in moves:
        dy, dx = directions[move]
        ry, rx = robot_position
        ny, nx = ry + dy, rx + dx

        # Check bounds and walls
        if ny < 0 or ny >= len(warehouse) or nx < 0 or nx + 1 >= len(warehouse[0]):
            continue
        if warehouse[ny][nx] == '#' or warehouse[ny][nx + 1] == '#':
            continue

        # Handle box movement
        if warehouse[ny][nx] == '[' and warehouse[ny][nx + 1] == ']':
            box_ny, box_nx = ny + dy, nx + dx

            # Check if the box can move
            if (
                box_ny < 0 or box_ny >= len(warehouse) or
                box_nx < 0 or box_nx + 1 >= len(warehouse[0]) or
                warehouse[box_ny][box_nx] in ['#', '['] or
                warehouse[box_ny][box_nx + 1] in ['#', ']']
            ):
                continue

            # Move the box
            warehouse[box_ny][box_nx] = '['
            warehouse[box_ny][box_nx + 1] = ']'
            warehouse[ny][nx] = '.'
            warehouse[ny][nx + 1] = '.'

        # Move the robot
        warehouse[ry][rx] = '.'
        warehouse[ry][rx + 1] = '.'
        warehouse[ny][nx] = '@'
        warehouse[ny][nx + 1] = '.'
        robot_position = (ny, nx)

    return warehouse


def print_scaled_warehouse(warehouse):
    for row in warehouse:
        print(''.join(row))


if __name__ == "__main__":
    input_file = "sample-input.txt"  # Replace with the path to your sample input file
    map_data, moves = read_input(input_file)
    warehouse, robot_position = parse_map(map_data)
    scaled_warehouse = scale_up_map(warehouse)

    print("\nInitial Scaled Warehouse State:")
    print_scaled_warehouse(scaled_warehouse)

    updated_warehouse = simulate_robot(scaled_warehouse, robot_position, moves)

    print("\nFinal Scaled Warehouse State:")
    print_scaled_warehouse(updated_warehouse)



Initial Scaled Warehouse State:
####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##....[]@.....[]..##
##[]##....[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################

Final Scaled Warehouse State:
####################
##....[][]..[][][]##
##............[][]##
##..............[]##
##[]............[]##
##[]##............##
##[]@...........[]##
##........[]....[]##
##[]..[][][][][]..##
####################


In [31]:
def read_input(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Split map and movement instructions
    map_data = []
    moves = ""
    is_map = True

    for line in lines:
        if line.strip() == "":
            continue
        if line.strip()[0] in "<>^v":
            is_map = False

        if is_map:
            map_data.append(line.rstrip())
        else:
            moves += line.strip()

    return map_data, moves


def parse_map(map_data):
    warehouse = []
    robot_position = None

    for y, row in enumerate(map_data):
        row_data = []
        for x, char in enumerate(row):
            if char == '@':
                robot_position = (y, x)
                row_data.append('@')  # Keep the robot in place for visualization
            else:
                row_data.append(char)
        warehouse.append(row_data)

    return warehouse, robot_position


def scale_up_map(warehouse):
    # Create a new scaled warehouse with dimensions 2x width and same height
    scaled_warehouse = [[" " for _ in range(len(warehouse[0]) * 2)] for _ in range(len(warehouse))]

    for y, row in enumerate(warehouse):
        for x, char in enumerate(row):
            # Map characters to their scaled counterparts
            if char == '#':
                scaled_warehouse[y][x * 2] = '#'
                scaled_warehouse[y][x * 2 + 1] = '#'
            elif char == 'O':
                scaled_warehouse[y][x * 2] = '['
                scaled_warehouse[y][x * 2 + 1] = ']'
            elif char == '.':
                scaled_warehouse[y][x * 2] = '.'
                scaled_warehouse[y][x * 2 + 1] = '.'
            elif char == '@':
                scaled_warehouse[y][x * 2] = '@'
                scaled_warehouse[y][x * 2 + 1] = '.'

    return scaled_warehouse


def simulate_robot(warehouse, robot_position, moves):
    directions = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -2),
        '>': (0, 2)
    }

    for move in moves:
        print("\nMove:", move)
        dy, dx = directions[move]
        ry, rx = robot_position
        ny, nx = ry + dy, rx + dx

        # Determine the range of tiles being pushed
        pushed_range = [(ry, rx)]
        if move in '<>':
            for i in range(1, 3):  # Include additional tiles in the push direction
                pushed_y, pushed_x = ry, rx + (i * dx // 2)
                if 0 <= pushed_x < len(warehouse[0]):
                    pushed_range.append((pushed_y, pushed_x))
        
        # Check if the push is blocked
        blocked = any(
            warehouse[py][px] in ['#', '[']
            for py, px in pushed_range
            if 0 <= px < len(warehouse[0])
        )
        if blocked:
            continue

        # Move the robot and boxes if applicable
        for py, px in reversed(pushed_range):
            next_py, next_px = py + dy, px + dx

            if (0 <= next_px < len(warehouse[0]) and
                    warehouse[py][px] in ['[', ']']):
                warehouse[next_py][next_px] = warehouse[py][px]
                warehouse[py][px] = '.'

        # Move the robot itself
        warehouse[ry][rx] = '.'
        warehouse[ny][nx] = '@'
        robot_position = (ny, nx)

        print_scaled_warehouse(warehouse)

    return warehouse


def print_scaled_warehouse(warehouse):
    for row in warehouse:
        print(''.join(row))


if __name__ == "__main__":
    input_file = "sample-input-part-002.txt"  # Replace with the path to your sample input file
    map_data, moves = read_input(input_file)
    warehouse, robot_position = parse_map(map_data)
    scaled_warehouse = scale_up_map(warehouse)

    print("\nInitial Scaled Warehouse State:")
    print_scaled_warehouse(scaled_warehouse)

    updated_warehouse = simulate_robot(scaled_warehouse, robot_position, moves)

    print("\nFinal Scaled Warehouse State:")
    print_scaled_warehouse(updated_warehouse)


Initial Scaled Warehouse State:
##############
##......##..##
##..........##
##....[][]@.##
##....[]....##
##..........##
##############

Move: <
##############
##......##..##
##..........##
##.@..[][]@.##
##....[]....##
##..........##
##############

Move: v
##############
##......##..##
##..........##
##....[][]@.##
##.@..[]....##
##..........##
##############

Move: v
##############
##......##..##
##..........##
##....[][]@.##
##....[]....##
##.@........##
##############

Move: <

Move: <

Move: ^
##############
##......##..##
##..........##
##....[][]@.##
##.@..[]....##
##..........##
##############

Move: ^
##############
##......##..##
##..........##
##.@..[][]@.##
##....[]....##
##..........##
##############

Move: <

Move: <

Move: ^
##############
##......##..##
##.@........##
##....[][]@.##
##....[]....##
##..........##
##############

Move: ^
##############
##.@....##..##
##..........##
##....[][]@.##
##....[]....##
##..........##
##############

Final Scaled Warehouse Stat

In [12]:
def solve():
    # Read the input file
    with open('sample-input.txt', 'r') as file:
        lines = [line.rstrip('\n') for line in file.readlines()]

    # Separate out the map portion vs. the moves portion
    warehouse_map = []
    move_lines = []
    parsing_map = True
    for line in lines:
        if parsing_map:
            if line.strip() == '' or not line.startswith('#'):
                parsing_map = False
            else:
                warehouse_map.append(line)
        else:
            move_lines.append(line.strip())

    # Combine all move-lines into one big string of moves
    moves = ''.join(move_lines)

    # Scale up the map
    scaled_map = []
    for row in warehouse_map:
        scaled_row = ''
        for ch in row:
            if ch == '#':
                scaled_row += '##'
            elif ch == 'O':
                scaled_row += '[]'
            elif ch == '.':
                scaled_row += '..'
            elif ch == '@':
                scaled_row += '@.'
        scaled_map.append(scaled_row)

    # Parse the scaled map into a data structure
    boxes = set()
    walls = set()
    robot = None
    for r, row in enumerate(scaled_map):
        for c, ch in enumerate(row):
            if ch == '#':
                walls.add((r, c))
            elif ch == '[':
                boxes.add((r, c))  # Left edge of wide box
            elif ch == '@':
                robot = (r, c)  # Left edge of robot

    # Helper for performing a move
    def attempt_move(robot_rc, boxes, dr, dc):
        (rr, cc) = robot_rc
        nr, nc = rr + dr, cc + dc

        # If new robot spot is a wall, move blocked, do nothing
        if (nr, nc) in walls:
            return robot_rc, boxes

        # If new robot spot is empty floor, just move
        if (nr, nc) not in boxes:
            return (nr, nc), boxes

        # Else, (nr,nc) has a box. We need to push that box chain if possible.
        chain = []
        check_r, check_c = nr, nc
        while (check_r, check_c) in boxes:
            chain.append((check_r, check_c))
            check_c += 2  # Move to the next wide box (2 columns wide)

        # Check the tile after the last box in the chain
        if (check_r, check_c) in walls or (check_r, check_c) in boxes:
            return robot_rc, boxes

        # Otherwise, we can push successfully
        new_boxes = set(boxes)
        for i in reversed(range(len(chain))):
            (br, bc) = chain[i]
            new_boxes.remove((br, bc))
            new_boxes.add((br + dr, bc + dc))

        new_robot = (nr, nc)
        return new_robot, new_boxes

    # Directions map
    dir_map = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -2),
        '>': (0, 2),
    }

    # Simulate all moves
    rpos = robot
    bset = boxes
    for m in moves:
        dr, dc = dir_map[m]
        rpos, bset = attempt_move(rpos, bset, dr, dc)

    # Compute final sum of GPS coordinates
    total = 0
    for (r, c) in bset:
        total += 100 * r + c

    print(total)

if __name__ == "__main__":
    solve()


7450


In [13]:
def main():
    # Read input from sample-input.txt
    with open("sample-input.txt", "r") as f:
        lines = [line.rstrip('\n') for line in f]

    # The puzzle states the first 10 lines are the map for this sample.
    # After these 10 lines, the rest are lines of moves (where newlines should be ignored).
    warehouse_map = lines[:10]
    move_lines = lines[10:]
    
    # Combine all move lines into one big string (ignore newlines).
    moves = ''.join(move_lines)

    # Parse the map into data structures:
    # - walls: set of (row, col) for '#'
    # - boxes: set of (row, col) for 'O'
    # - robot: (row, col) for '@'
    walls = set()
    boxes = set()
    robot = None
    height = len(warehouse_map)
    width = len(warehouse_map[0]) if height > 0 else 0

    for r, row_str in enumerate(warehouse_map):
        for c, ch in enumerate(row_str):
            if ch == '#':
                walls.add((r, c))
            elif ch == 'O':
                boxes.add((r, c))
            elif ch == '@':
                robot = (r, c)

    # Directions map
    dir_map = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1),
    }

    def attempt_move(robot_rc, boxes, dr, dc):
        """Attempt to move the robot from robot_rc in direction (dr, dc).
           Returns updated (robot_rc, boxes)."""
        (rr, cc) = robot_rc
        nr, nc = rr + dr, cc + dc  # proposed new robot position
        
        # If the new spot is a wall, do nothing
        if (nr, nc) in walls:
            return robot_rc, boxes
        
        # If the new spot is empty, robot just moves
        if (nr, nc) not in boxes:
            return (nr, nc), boxes
        
        # Otherwise, the new spot has a box. Attempt to push the chain of boxes.
        chain = []
        check_r, check_c = nr, nc
        
        # Collect consecutive boxes in this direction
        while (check_r, check_c) in boxes:
            chain.append((check_r, check_c))
            check_r += dr
            check_c += dc
        
        # Now (check_r, check_c) is the cell after the last box in the chain
        # If that cell is a wall or a box, push fails
        if (check_r, check_c) in walls or (check_r, check_c) in boxes:
            return robot_rc, boxes
        
        # If we get here, we can push all boxes in 'chain' forward by one cell
        new_boxes = set(boxes)
        for i in reversed(range(len(chain))):
            (br, bc) = chain[i]
            new_boxes.remove((br, bc))
            new_boxes.add((br + dr, bc + dc))
        
        # Robot steps into the position vacated by the first box
        new_robot = (nr, nc)
        return new_robot, new_boxes

    # Run the simulation over all moves
    rpos = robot
    bset = boxes
    for m in moves:
        if m not in dir_map:
            continue  # Just skip any unexpected character
        dr, dc = dir_map[m]
        rpos, bset = attempt_move(rpos, bset, dr, dc)

    # Compute final sum of GPS coordinates
    # GPS(row, col) = 100 * row + col
    total_gps = sum(100*r + c for (r, c) in bset)
    print(total_gps)

if __name__ == "__main__":
    main()

10092


In [14]:
def solve():
    # Read the input file
    with open('sample-input.txt', 'r') as file:
        lines = [line.rstrip('\n') for line in file.readlines()]

    # Separate out the map portion vs. the moves portion
    warehouse_map = []
    move_lines = []
    parsing_map = True
    for line in lines:
        if parsing_map:
            if line.strip() == '' or not line.startswith('#'):
                parsing_map = False
            else:
                warehouse_map.append(line)
        else:
            move_lines.append(line.strip())

    # Combine all move-lines into one big string of moves
    moves = ''.join(move_lines)

    # Scale up the map
    scaled_map = []
    for row in warehouse_map:
        scaled_row = ''
        for ch in row:
            if ch == '#':
                scaled_row += '##'
            elif ch == 'O':
                scaled_row += '[]'
            elif ch == '.':
                scaled_row += '..'
            elif ch == '@':
                scaled_row += '@.'
        scaled_map.append(scaled_row)

    # Parse the scaled map into a data structure
    boxes = set()
    walls = set()
    robot = None
    for r, row in enumerate(scaled_map):
        for c in range(0, len(row), 2):  # Iterate two columns at a time
            ch = row[c:c+2]
            if ch == '##':
                walls.add((r, c))
            elif ch == '[]':
                boxes.add((r, c))  # Left edge of wide box
            elif ch == '@.':
                robot = (r, c)  # Left edge of robot

    # Helper for performing a move
    def attempt_move(robot_rc, boxes, dr, dc):
        (rr, cc) = robot_rc
        nr, nc = rr + dr, cc + dc * 2  # Adjust for two-column-wide cells

        # If new robot spot is a wall, move blocked, do nothing
        if (nr, nc) in walls:
            return robot_rc, boxes

        # If new robot spot is empty floor, just move
        if (nr, nc) not in boxes:
            return (nr, nc), boxes

        # Else, (nr,nc) has a box. We need to push that box chain if possible.
        chain = []
        check_r, check_c = nr, nc
        while (check_r, check_c) in boxes:
            chain.append((check_r, check_c))
            check_c += 2  # Move to the next wide box (2 columns wide)

        # Check the tile after the last box in the chain
        if (check_r, check_c) in walls or (check_r, check_c) in boxes:
            return robot_rc, boxes

        # Otherwise, we can push successfully
        new_boxes = set(boxes)
        for i in reversed(range(len(chain))):
            (br, bc) = chain[i]
            new_boxes.remove((br, bc))
            new_boxes.add((br + dr, bc + dc * 2))

        new_robot = (nr, nc)
        return new_robot, new_boxes

    # Directions map
    dir_map = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1),
    }

    # Simulate all moves
    rpos = robot
    bset = boxes
    for m in moves:
        dr, dc = dir_map[m]
        rpos, bset = attempt_move(rpos, bset, dr, dc)

    # Compute final sum of GPS coordinates
    total = 0
    for (r, c) in bset:
        total += 100 * r + c // 2  # Adjust column for wide box

    print(total)

if __name__ == "__main__":
    solve()


7375


In [15]:
def read_input(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Split map and movement instructions
    map_data = []
    moves = ""
    is_map = True

    for line in lines:
        if line.strip().startswith("<") or line.strip().startswith(">") or 
           line.strip().startswith("^") or line.strip().startswith("v"):
            is_map = False
        
        if is_map:
            map_data.append(line.strip())
        else:
            moves += line.strip()

    return map_data, moves


def parse_map(map_data):
    warehouse = []
    robot_position = None

    for y, row in enumerate(map_data):
        row_data = []
        for x, char in enumerate(row):
            if char == '@':
                robot_position = (y, x)
                row_data.append('.')
            else:
                row_data.append(char)
        warehouse.append(row_data)

    return warehouse, robot_position


def simulate_movements(warehouse, robot_pos, moves):
    directions = {'^': (-1, 0), 'v': (1, 0), '<': (0, -1), '>': (0, 1)}
    
    for move in moves:
        dy, dx = directions[move]
        ny, nx = robot_pos[0] + dy, robot_pos[1] + dx

        if warehouse[ny][nx] == '#':
            continue  # Wall, cannot move

        if warehouse[ny][nx] == 'O':
            box_new_y, box_new_x = ny + dy, nx + dx

            if warehouse[box_new_y][box_new_x] in ['.', ' ']:
                # Move box and robot
                warehouse[box_new_y][box_new_x] = 'O'
                warehouse[ny][nx] = '.'
                robot_pos = (ny, nx)

        elif warehouse[ny][nx] in ['.', ' ']:
            # Move robot only
            robot_pos = (ny, nx)

    return warehouse


def calculate_gps_coordinates(warehouse):
    gps_sum = 0

    for y, row in enumerate(warehouse):
        for x, char in enumerate(row):
            if char == 'O':
                gps_sum += 100 * y + x

    return gps_sum


if __name__ == "__main__":
    input_file = "input.txt"  # Replace with the path to your input file
    map_data, moves = read_input(input_file)
    warehouse, robot_position = parse_map(map_data)
    warehouse = simulate_movements(warehouse, robot_position, moves)
    result = calculate_gps_coordinates(warehouse)
    print("Sum of GPS coordinates:", result)

SyntaxError: invalid syntax (294309590.py, line 11)